# Goal: generating some matchings and attempt to code up Gale-Shapley 

In [31]:
import import_ipynb 
import test_stable as test 
import cases as c
import random

In [32]:
import random 

def sim_gs_1to1(num_propose, num_receive): #generate random input for 1-to-1 Gale-Shapley
    rank_proposer, rank_receiver = {}, {}
    receiver_set = ["R" + str(i) for i in range(1,num_receive+1)] #set of receiver 
    proposer_set = ["P" + str(i) for i in range(1,num_propose+1)] #set of proposers
    for i in range(1,num_propose+1):
        random.shuffle(receiver_set) #randomize the list
        rank_proposer["P" + str(i)] = receiver_set.copy() #add 
    for i in range(1,num_receive +1): #similarly 
        random.shuffle(proposer_set)
        rank_receiver["R" + str(i)] = proposer_set.copy()
    return rank_proposer, rank_receiver

print(sim_gs_1to1(4,5))

({'P1': ['R3', 'R2', 'R5', 'R4', 'R1'], 'P2': ['R1', 'R3', 'R4', 'R5', 'R2'], 'P3': ['R2', 'R1', 'R5', 'R3', 'R4'], 'P4': ['R5', 'R1', 'R2', 'R4', 'R3']}, {'R1': ['P3', 'P2', 'P1', 'P4'], 'R2': ['P4', 'P1', 'P3', 'P2'], 'R3': ['P3', 'P4', 'P2', 'P1'], 'R4': ['P1', 'P3', 'P2', 'P4'], 'R5': ['P2', 'P3', 'P1', 'P4']})


In [51]:
def gale_shapley11(proposers_rank, receivers_rank):
    #basically doing deep copy so we dont modify the original rankings
    proposers = {key: value[:] for key, value in proposers_rank.items()}  # Copying the lists in proposers_rank
    receivers = {key: value[:] for key, value in receivers_rank.items()}  # Copying the lists in receivers_rank
    unmatched = {"boy": [i for i in proposers]}
    engagement = {} # (girl: boy)
    day = 0
    rejects = {boy: [] for boy in proposers}
    while len(unmatched["boy"]) > 0: #until every man is engaged 
        propositions = {}
        # print("engagement: ", engagement, "unmatched: ", unmatched)
        # round 1: boy's turn to propose 
        for unmatched_boy in unmatched["boy"]:
            highest_girl = proposers[unmatched_boy].pop(0)
            # if already rejected before, then go down the list
            while highest_girl in rejects[unmatched_boy]: 
                highest_girl = proposers[unmatched_boy].pop(0)
            if highest_girl not in propositions:
                propositions[highest_girl] = [unmatched_boy]
            else:
                propositions[highest_girl].append(unmatched_boy)
        # print("propositions: ", propositions)
        # now we have a list of propositions
        # round 2: girl's turn to choose 
        for proposed_girl in propositions:
            considerations = receivers[proposed_girl].copy() #temporary copy 
            highest_boy = considerations.pop(0) 
            while highest_boy not in propositions[proposed_girl]: #going down the list
                #to the highest-ranked proposing boy 
                highest_boy = considerations.pop(0) 
            # assume we have a match 
            if proposed_girl not in engagement: #the girl not engaged - then matched
                engagement[proposed_girl] = highest_boy
                unmatched["boy"].remove(highest_boy) #remove from bachelor pool
            else: #if engaged, then consider
                current = engagement[proposed_girl]
                alternative = highest_boy
                ranking = receivers[proposed_girl]
                if ranking.index(alternative) < ranking.index(current): 
                    #if the alternative is better: break up with current engagement 
                    # broken-with boy is back in the market 
                    unmatched["boy"].append(current)
                    # print(proposed_girl, "broke up with", current, "for", alternative)
                    rejects[current].append(proposed_girl)
                    unmatched["boy"].remove(alternative) #remove from bachelor pool
                    engagement[proposed_girl] = alternative
                else:
                    # print(proposed_girl, "rejects", alternative, "and stays with", current) 
                    rejects[alternative].append(proposed_girl)
            day += 1
    # print("final engagement: ", engagement, "it took", day, "days")
    marriage = [(proposer, receiver) for proposer, receiver in engagement.items()]
    return marriage 


In [68]:
# simulate and solve Gale-Shapeley matching 
rank_propose, rank_receiver = sim_gs_1to1(7,7)
print(rank_propose, rank_receiver)
matching = gale_shapley11(rank_propose, rank_receiver)
print(matching)
test.test_stable(rank_propose, rank_receiver, matching)

# for some reasons the algorithm generalizes only for when n_proposer < n_receiver 
# but not for when n_receiver is smaller 
# investigate this 


{'P1': ['R6', 'R3', 'R2', 'R7', 'R5', 'R4', 'R1'], 'P2': ['R1', 'R4', 'R6', 'R3', 'R7', 'R5', 'R2'], 'P3': ['R5', 'R7', 'R2', 'R3', 'R6', 'R1', 'R4'], 'P4': ['R5', 'R3', 'R7', 'R1', 'R6', 'R2', 'R4'], 'P5': ['R7', 'R1', 'R4', 'R3', 'R5', 'R6', 'R2'], 'P6': ['R4', 'R3', 'R1', 'R2', 'R7', 'R6', 'R5'], 'P7': ['R3', 'R1', 'R6', 'R2', 'R5', 'R7', 'R4']} {'R1': ['P6', 'P4', 'P1', 'P5', 'P7', 'P2', 'P3'], 'R2': ['P2', 'P6', 'P5', 'P7', 'P4', 'P1', 'P3'], 'R3': ['P1', 'P3', 'P7', 'P2', 'P5', 'P6', 'P4'], 'R4': ['P4', 'P7', 'P2', 'P5', 'P6', 'P1', 'P3'], 'R5': ['P4', 'P7', 'P1', 'P2', 'P5', 'P6', 'P3'], 'R6': ['P3', 'P6', 'P1', 'P7', 'P4', 'P2', 'P5'], 'R7': ['P2', 'P5', 'P1', 'P7', 'P3', 'P4', 'P6']}
[('R6', 'P1'), ('R1', 'P2'), ('R5', 'P4'), ('R7', 'P5'), ('R4', 'P6'), ('R3', 'P7'), ('R2', 'P3')]
The matching [('R6', 'P1'), ('R1', 'P2'), ('R5', 'P4'), ('R7', 'P5'), ('R4', 'P6'), ('R3', 'P7'), ('R2', 'P3')] is stable


In [35]:
# # # let's attempt to code up a 1-to-1 matching 

# # I THINK THIS IS THE BOSTON MATCHING ALGORITHM! 


# # # reverse the ranking so that we have a stack for each 
# # for girl in girl_rank:
# #     girl_rank[girl].reverse()
# # for boy in boy_rank:
# #     boy_rank[boy].reverse()

# print("girl rank", girl_rank)
# print("boy rank", boy_rank)
# #now we will attempt to code up a matching system
# matches = [] #format: (girl, boy)

# no_days = 3

# #list of unmatched people
# unmatched = {"girl": [i for i in girl_rank],"boy": [i for i in boy_rank]}

# day = 0 
# while len(unmatched["girl"]) + len(unmatched["boy"]) > 0: #while still unmatched people
#     propositions = {}
#     print(boy_rank, girl_rank)
#     # S1: boy's turn to propose 
#     for unmatched_boy in unmatched["boy"]:
#         highest_girl = boy_rank[unmatched_boy].pop()
#         while highest_girl not in unmatched["girl"]: #going down to list
#             #to the first unmatched girl 
#             highest_girl = boy_rank[unmatched_boy].pop()
#         propositions[unmatched_boy] = highest_girl
#     print("propositions", propositions)
#     # now we have a list of propositions of boy to their current highested-ranked girls 
#     # S2: girl's turn to accept 
#     for unmatched_girl in unmatched["girl"]: #for each unmatched girl 
#         highest_boy = girl_rank[unmatched_girl].pop()
#         while highest_boy not in unmatched["boy"]: #going down the list
#             #to the first unmatched girl 
#             highest_boy = girl_rank[unmatched_girl].pop()
#         if highest_boy in propositions and propositions[highest_boy] == unmatched_girl: #if the boy they want is proposing
#             a_match = (propositions[highest_boy], highest_boy)
#             matches.append(a_match)
#             print("found", a_match)
#             unmatched["girl"].remove(a_match[0])
#             unmatched["boy"].remove(a_match[1])
#             del boy_rank[highest_boy]
#             del girl_rank[propositions[highest_boy]]
#     # S3: restore the ranking for both if they are not matched
#     for proposition in propositions: #restore for the boys
#         if propositions[proposition] in unmatched["girl"]: #still unmatched
#             boy_rank[proposition].append(propositions[proposition])
#     # print("unmatched", unmatched)
#     day += 1
#     # now we have a list of propositions

# print("stable matches found:", matches)
# print("number of days needed", day)
